# Análisis Exploratorio de Datos (EDA) - Online Retail

En este cuaderno, realizaremos un **análisis paso a paso** de un dataset de facturación que contiene columnas como:
 - **InvoiceNo**: Número de factura
 - **StockCode**: Código del producto
 - **Description**: Descripción del producto
 - **Quantity**: Cantidad vendida (puede ser negativa en devoluciones)
 - **InvoiceDate**: Fecha de la factura
 - **UnitPrice**: Precio unitario de cada artículo
 - **CustomerID**: ID del cliente (puede haber nulos si no se registró)
 - **Country**: País donde se realizó la compra

El objetivo es **entender** el comportamiento de las ventas, los países más relevantes, los productos más comprados, y otras métricas de interés.

In [1]:
!wget https://github.com/javierherrera1996/lecture_analytics/raw/main/datasets/ecomerce_customers_segmentation.zip
!unzip ecomerce_customers_segmentation.zip

--2025-09-10 23:34:04--  https://github.com/javierherrera1996/lecture_analytics/raw/main/datasets/ecomerce_customers_segmentation.zip
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/javierherrera1996/lecture_analytics/main/datasets/ecomerce_customers_segmentation.zip [following]
--2025-09-10 23:34:04--  https://raw.githubusercontent.com/javierherrera1996/lecture_analytics/main/datasets/ecomerce_customers_segmentation.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7548686 (7.2M) [application/zip]
Saving to: ‘ecomerce_customers_segmentation.zip’

ecomerce_customers_ 100%[===================>]

## 1. Importar librerías y cargar datos
Asegúrate de tener el archivo CSV en la misma carpeta, o ajusta la ruta de `pd.read_csv` según tu necesidad.

In [3]:
import pandas as pd
import numpy as np

# Carga del dataset (ajusta el nombre a tu archivo)
df = pd.read_csv('data.csv',encoding="latin-1") # Puedes necesitar encoding='latin1' si hay caracteres especiales
pd.set_option('display.max_columns', None)
# Mostramos las primeras filas para inspección
df.head(15)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,12/1/2010 8:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,12/1/2010 8:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,12/1/2010 8:26,3.39,17850.0,United Kingdom
5,536365,22752,SET 7 BABUSHKA NESTING BOXES,2,12/1/2010 8:26,7.65,17850.0,United Kingdom
6,536365,21730,GLASS STAR FROSTED T-LIGHT HOLDER,6,12/1/2010 8:26,4.25,17850.0,United Kingdom
7,536366,22633,HAND WARMER UNION JACK,6,12/1/2010 8:28,1.85,17850.0,United Kingdom
8,536366,22632,HAND WARMER RED POLKA DOT,6,12/1/2010 8:28,1.85,17850.0,United Kingdom
9,536367,84879,ASSORTED COLOUR BIRD ORNAMENT,32,12/1/2010 8:34,1.69,13047.0,United Kingdom


## 2. Información general del dataset
Aquí veremos:
- Dimensiones del DataFrame
- Tipos de datos
- Presencia de valores nulos
- Estadísticos descriptivos de las columnas numéricas

In [20]:
df.shape


(541909, 8)

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 8 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   InvoiceNo    541909 non-null  object        
 1   StockCode    541909 non-null  object        
 2   Description  540455 non-null  object        
 3   Quantity     541909 non-null  int64         
 4   InvoiceDate  541909 non-null  datetime64[ns]
 5   UnitPrice    541909 non-null  float64       
 6   CustomerID   406829 non-null  float64       
 7   Country      541909 non-null  object        
dtypes: datetime64[ns](1), float64(2), int64(1), object(4)
memory usage: 33.1+ MB


In [23]:
df.describe()

,Quantity,InvoiceDate,UnitPrice,CustomerID
count,541909.000000,541909,541909.000000,406829.000000
mean,9.552250,2011-07-04 13:34:57.156386048,4.611114,15287.690570
min,-80995.000000,2010-12-01 08:26:00,-11062.060000,12346.000000
25%,1.000000,2011-03-28 11:34:00,1.250000,13953.000000
50%,3.000000,2011-07-19 17:17:00,2.080000,15152.000000
75%,10.000000,2011-10-19 11:27:00,4.130000,16791.000000
max,80995.000000,2011-12-09 12:50:00,38970.000000,18287.000000
std,218.081158,NaN,96.759853,1713.600303


In [25]:
df.isnull().sum()

,0
InvoiceNo,0
StockCode,0
Description,1454
Quantity,0
InvoiceDate,0
UnitPrice,0
CustomerID,135080
Country,0


**Observación**: Algunas facturas pueden ser devoluciones si `Quantity` es negativa. Además, puede haber filas sin `CustomerID`.

## 3. Limpieza básica y transformación
### 3.1. Revisar y eliminar posibles filas con datos problemáticos
 - Filas donde `Description` esté vacía
 - Filas con `Quantity == 0` (puede haber poco sentido en un registro así)
 - Filas con `UnitPrice <= 0` (podría ser error o promoción anómala)
 - Convertir la columna `InvoiceDate` a formato fecha, para luego analizar tiempos

In [32]:
df.dropna(subset=['Description'])

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


from matplotlib import pyplot as plt
_df_0['Quantity'].plot(kind='hist', bins=20, title='Quantity')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['UnitPrice'].plot(kind='hist', bins=20, title='UnitPrice')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2['CustomerID'].plot(kind='hist', bins=20, title='CustomerID')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3.plot(kind='scatter', x='Quantity', y='UnitPrice', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

...

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['InvoiceDate']
  ys = series['CustomerID']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_7.sort_values('InvoiceDate', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('InvoiceDate')
_ = plt.ylabel('CustomerID')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['InvoiceDate']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'InvoiceDate'}, axis=1)
              .sort_values('InvoiceDate', ascending=True))
  xs = counted['InvoiceDate']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_8.sort_values('InvoiceDate', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('InvoiceDate')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_9['Quantity'].plot(kind='line', figsize=(8, 4), title='Quantity')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_10['UnitPrice'].plot(kind='line', figsize=(8, 4), title='UnitPrice')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_11['CustomerID'].plot(kind='line', figsize=(8, 4), title='CustomerID')
plt.gca().spines[['top', 'right']].set_visible(False)

In [75]:
df=df[df['Quantity']!=0]

In [39]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


In [48]:
df = df[df['UnitPrice']> 0]

In [42]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France


## 4. Creación de columna "TotalPrice"
Para conocer el valor total de cada línea de venta, calculamos `Quantity * UnitPrice`.

In [55]:
df = df.assign(TotalPrice=df['Quantity'] * df['UnitPrice'])

In [56]:
df

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
...,...,...,...,...,...,...,...,...,...
541904,581587,22613,PACK OF 20 SPACEBOY NAPKINS,12,2011-12-09 12:50:00,0.85,12680.0,France,10.20
541905,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France,12.60
541906,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60
541907,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France,16.60


## 5. Análisis paso a paso
A continuación, veremos distintos análisis que **paso a paso** nos ayudarán a entender mejor el negocio.

### 5.1. ¿Cuántas transacciones totales hay y cuántos clientes diferentes?
Esto da un primer vistazo al tamaño del negocio en términos de facturas y base de clientes.

In [61]:
# Total number of transactions (rows)
total_transactions = len(df)

# Number of unique customers (excluding NaNs in CustomerID)
unique_customers = df['CustomerID'].nunique()

print(f"Total number of transactions: {total_transactions}")
print(f"Number of unique customers: {unique_customers}")

Total number of transactions: 530104
Number of unique customers: 4338


### 5.2. ¿Cuáles son los países más frecuentes en las ventas?
Podemos contar cuántas líneas de factura vienen de cada país. Esto ayuda a entender nuestro alcance geográfico.

In [68]:
df.groupby('Country').count()


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,TotalPrice
Country,,,,,,,,
Australia,1182,1182,1182,1182,1182,1182,1182,1182
Austria,398,398,398,398,398,398,398,398
Bahrain,18,18,18,18,18,18,17,18
Belgium,2031,2031,2031,2031,2031,2031,2031,2031
Brazil,32,32,32,32,32,32,32,32
Canada,151,151,151,151,151,151,151,151
Channel Islands,748,748,748,748,748,748,748,748
Cyprus,614,614,614,614,614,614,614,614
Czech Republic,25,25,25,25,25,25,25,25


In [76]:

Paises_Con_ventas_mas_frecuentes = df.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False)

display(Paises_Con_ventas_mas_frecuentes)

,TotalPrice
Country,
United Kingdom,9025222.084
Netherlands,285446.340
EIRE,283453.960
Germany,228867.140
France,209715.110
Australia,138521.310
Spain,61577.110
Switzerland,57089.900
Belgium,41196.340


> Tip: Si queremos ver el total en **monetario** por país, podemos agrupar por `Country` y sumar `TotalPrice`.

In [79]:
df.groupby('Country')['TotalPrice'].sum().sort_values(ascending=False)

,TotalPrice
Country,
United Kingdom,9025222.084
Netherlands,285446.340
EIRE,283453.960
Germany,228867.140
France,209715.110
Australia,138521.310
Spain,61577.110
Switzerland,57089.900
Belgium,41196.340


### 5.3. ¿Cuáles son los productos más vendidos?
Aquí vamos a usar `Description` para ver cuántas líneas se han vendido (o la suma total de `Quantity`).

In [80]:
# Top 10 productos por cantidad total vendidos
df.groupby('Description')['Quantity'].sum().sort_values(ascending=False).head(10)

,Quantity
Description,
"PAPER CRAFT , LITTLE BIRDIE",80995
MEDIUM CERAMIC TOP STORAGE JAR,78033
WORLD WAR 2 GLIDERS ASSTD DESIGNS,55047
JUMBO BAG RED RETROSPOT,48474
WHITE HANGING HEART T-LIGHT HOLDER,37891
POPCORN HOLDER,36761
ASSORTED COLOUR BIRD ORNAMENT,36461
PACK OF 72 RETROSPOT CAKE CASES,36419
RABBIT NIGHT LIGHT,30788


### 5.4. ¿Cuál es el total de ingresos (revenue) y el ticket promedio?
 - El ingreso total es la suma de **TotalPrice**.
 - El ticket promedio podemos calcularlo como el promedio de **TotalPrice** dentro de cada factura (o línea).

In [88]:
total_income = df['TotalPrice'].sum()

In [117]:
print(total_income)

10666684.544


> Para un ticket promedio a nivel de **factura**, debemos agrupar por `InvoiceNo`, sumar `TotalPrice` y luego promediar.

In [115]:
Ticket_promedio_por_factura=df.groupby("InvoiceNo")['TotalPrice'].sum().mean()
print(Ticket_promedio_por_factura)

534.403033266533


### 5.6. ¿Tenemos devoluciones?
Como `Quantity` puede ser negativa, **representa** devoluciones. Veamos cuántas hay y el impacto en valor.

In [111]:
# Filtrar devoluciones (Quantity < 0)
devoluciones_df = df[df['Quantity'] < 0]

print("DataFramewith devoluciones:")
display(devoluciones_df.head(10))

DataFramewith devoluciones:


,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPrice


### 5.7. ¿Cuántos clientes realizan compras repetidas?
Podemos ver cuántos clientes compran más de una vez.

In [116]:
df.groupby('CustomerID')['InvoiceNo'].nunique().sort_values(ascending=False)

,InvoiceNo
CustomerID,
12748.0,209
14911.0,201
17841.0,124
13089.0,97
14606.0,93
...,...
18213.0,1
18215.0,1
18269.0,1


## 6. Posibles siguientes pasos
- **Crear segmentaciones** (p. ej. RFM: Recency, Frequency, Monetary) para clasificar clientes.
- **Analizar descuentos/promos** si tuvieras columnas extra.
- **Fusionar** con datos de marketing para calcular CAC (Customer Acquisition Cost) y LTV (Lifetime Value), etc.
- **Explorar** variables estacionales o días de la semana con mayor venta.

Con este **análisis paso a paso**, se obtiene un primer panorama de:
 - Cantidad de facturas y clientes.
 - Productos más vendidos.
 - Ingresos por país.
 - Montos totales y tickets promedio.
 - Distribución mensual de ventas.
 - Devoluciones y recurrencia de clientes.

¡A medida que se adquieran más habilidades en Python y análisis de datos, se pueden añadir gráficos, segmentaciones avanzadas y modelos predictivos!